# Project 3

In this project, you will perform a logistic regression on the admissions data we've been working with in projects 1 and 2.

In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import pandas as pd
import statsmodels.api as sm
import pylab as pl
import numpy as np


In [2]:
df_raw = pd.read_csv("../assets/admissions.csv")
df = df_raw.dropna() 
print df.head()

   admit    gre   gpa  prestige
0      0  380.0  3.61       3.0
1      1  660.0  3.67       3.0
2      1  800.0  4.00       1.0
3      1  640.0  3.19       4.0
4      0  520.0  2.93       4.0


## Part 1. Frequency Tables

#### 1. Let's create a frequency table of our variables

In [3]:
# frequency table for prestige and whether or not someone was admitted
pd.crosstab(df['prestige'], df['admit'])

admit,0,1
prestige,,
1.0,28,33
2.0,95,53
3.0,93,28
4.0,55,12


## Part 2. Return of dummy variables

#### 2.1 Create class or dummy variables for prestige 

In [11]:
dummy_ranks= pd.get_dummies(df['prestige'], prefix='prestige')
new_df = df.join(dummy_ranks)
new_df.head()

,admit,gre,gpa,prestige,prestige_1.0,prestige_2.0,prestige_3.0,prestige_4.0
0,0,380.0,3.61,3.0,0.0,0.0,1.0,0.0
1,1,660.0,3.67,3.0,0.0,0.0,1.0,0.0
2,1,800.0,4.00,1.0,1.0,0.0,0.0,0.0
3,1,640.0,3.19,4.0,0.0,0.0,0.0,1.0
4,0,520.0,2.93,4.0,0.0,0.0,0.0,1.0


#### 2.2 When modeling our class variables, how many do we need? 



Answer: 

## Part 3. Hand calculating odds ratios

Develop your intuition about expected outcomes by hand calculating odds ratios.

In [12]:
cols_to_keep = ['admit', 'gre', 'gpa']
handCalc = df[cols_to_keep].join(dummy_ranks.ix[:, 'prestige_1':])
print handCalc.head()

   admit    gre   gpa  prestige_1.0  prestige_2.0  prestige_3.0  prestige_4.0
0      0  380.0  3.61           0.0           0.0           1.0           0.0
1      1  660.0  3.67           0.0           0.0           1.0           0.0
2      1  800.0  4.00           1.0           0.0           0.0           0.0
3      1  640.0  3.19           0.0           0.0           0.0           1.0
4      0  520.0  2.93           0.0           0.0           0.0           1.0


In [14]:
#crosstab prestige 1 admission 
# frequency table cutting prestige and whether or not someone was admitted

def freqtable (dummy_ranks):
    return pd.crosstab(handCalc[dummy_ranks], handCalc['admit'])
    print

for rank in dummy_ranks:
    print freqtable(rank)


admit           0   1
prestige_1.0         
0.0           243  93
1.0            28  33
admit           0   1
prestige_2.0         
0.0           176  73
1.0            95  53
admit           0   1
prestige_3.0         
0.0           178  98
1.0            93  28
admit           0    1
prestige_4.0          
0.0           216  114
1.0            55   12


#### 3.1 Use the cross tab above to calculate the odds of being admitted to grad school if you attended a #1 ranked college

In [21]:
prestige_1_odds = float(33)/61
print prestige_1_odds

0.540983606557


#### 3.2 Now calculate the odds of admission if you did not attend a #1 ranked college

In [23]:
prestige_non_1_odds = float(53+28+12)/(95+53+93+28+55+12)
print prestige_non_1_odds

0.276785714286


#### 3.3 Calculate the odds ratio

In [25]:
(33 / 28.0) / (93 / 243.0)

3.079493087557604

#### 3.4 Write this finding in a sentenance: 

Answer: A student from a top ranked school is 3 times more likely to be admitted to UCLA than students from lower ranked schools. 


#### 3.5 Print the cross tab for prestige_4

In [30]:
pd.crosstab(new_df['prestige_4.0'], new_df['admit'])

admit,0,1
prestige_4.0,,
0.0,216,114
1.0,55,12


#### 3.6 Calculate the OR 

In [32]:
(12/114.0) / (55/216.0)

0.4133971291866028

#### 3.7 Write this finding in a sentence

Answer: When a student from a lower ranked school applies to UCLA their probability of admission to UCLA decreases by 59% when compare to the probability of a student from a top ranked school.

## Part 4. Analysis

In [33]:
# create a clean data frame for the regression
cols_to_keep = ['admit', 'gre', 'gpa']
data = df[cols_to_keep].join(dummy_ranks.ix[:, 'prestige_2':])
print data.head()

   admit    gre   gpa  prestige_2.0  prestige_3.0  prestige_4.0
0      0  380.0  3.61           0.0           1.0           0.0
1      1  660.0  3.67           0.0           1.0           0.0
2      1  800.0  4.00           0.0           0.0           0.0
3      1  640.0  3.19           0.0           0.0           1.0
4      0  520.0  2.93           0.0           0.0           1.0


We're going to add a constant term for our Logistic Regression. The statsmodels function we're going to be using requires that intercepts/constants are specified explicitly.

In [34]:
# manually add the intercept
data['intercept'] = 1.0

#### 4.1 Set the covariates to a variable called train_cols

In [35]:
train_cols = ['gre', 'gpa', 'prestige_2.0', 'prestige_3.0', 'prestige_4.0']

#### 4.2 Fit the model

In [39]:
model = sm.Logit(data['admit'], data[train_cols])
score = model.fit()

Optimization terminated successfully.
         Current function value: 0.589121
         Iterations 5


#### 4.3 Print the summary results

In [45]:
print score.summary()

                           Logit Regression Results                           
Dep. Variable:                  admit   No. Observations:                  397
Model:                          Logit   Df Residuals:                      392
Method:                           MLE   Df Model:                            4
Date:                Mon, 25 Jul 2016   Pseudo R-squ.:                 0.05722
Time:                        14:22:35   Log-Likelihood:                -233.88
converged:                       True   LL-Null:                       -248.08
                                        LLR p-value:                 1.039e-05
                   coef    std err          z      P>|z|      [95.0% Conf. Int.]
--------------------------------------------------------------------------------
gre              0.0014      0.001      1.308      0.191        -0.001     0.003
gpa             -0.1323      0.195     -0.680      0.497        -0.514     0.249
prestige_2.0    -0.9562      0.302     -3.17

#### 4.4 Calculate the odds ratios of the coeffiencents and their 95% CI intervals

hint 1: np.exp(X)

hint 2: conf['OR'] = params
        
           conf.columns = ['2.5%', '97.5%', 'OR']

In [43]:
np.exp(score.params)

gre             1.001368
gpa             0.876073
prestige_2.0    0.384342
prestige_3.0    0.214918
prestige_4.0    0.154135
dtype: float64

#### 4.5 Interpret the OR of Prestige_2

Answer: A student from a prestige 2 school has a 38% chance of being admitted to UCLA when comparing to other prestige schools, holding all other variables constant.  

#### 4.6 Interpret the OR of GPA

Answer: Students with high GPA scores have a 87% chance of being admitted to UCLA holding all other variables constant. 

## Part 5: Predicted probablities


As a way of evaluating our classifier, we're going to recreate the dataset with every logical combination of input values. This will allow us to see how the predicted probability of admission increases/decreases across different variables. First we're going to generate the combinations using a helper function called cartesian (above).

We're going to use np.linspace to create a range of values for "gre" and "gpa". This creates a range of linearly spaced values from a specified min and maximum value--in our case just the min/max observed values.

In [52]:
def cartesian(arrays, out=None):
    """
    Generate a cartesian product of input arrays.
    Parameters
    ----------
    arrays : list of array-like
        1-D arrays to form the cartesian product of.
    out : ndarray
        Array to place the cartesian product in.
    Returns
    -------
    out : ndarray
        2-D array of shape (M, len(arrays)) containing cartesian products
        formed of input arrays.
    Examples
    --------
    >>> cartesian(([1, 2, 3], [4, 5], [6, 7]))
    array([[1, 4, 6],
           [1, 4, 7],
           [1, 5, 6],
           [1, 5, 7],
           [2, 4, 6],
           [2, 4, 7],
           [2, 5, 6],
           [2, 5, 7],
           [3, 4, 6],
           [3, 4, 7],
           [3, 5, 6],
           [3, 5, 7]])
    """

    arrays = [np.asarray(x) for x in arrays]
    dtype = arrays[0].dtype

    n = np.prod([x.size for x in arrays])
    if out is None:
        out = np.zeros([n, len(arrays)], dtype=dtype)

    m = n / arrays[0].size
    out[:,0] = np.repeat(arrays[0], m)
    if arrays[1:]:
        cartesian(arrays[1:], out=out[0:m,1:])
        for j in xrange(1, arrays[0].size):
            out[j*m:(j+1)*m,1:] = out[0:m,1:]
    return out

In [69]:
# instead of generating all possible values of GRE and GPA, we're going
# to use an evenly spaced range of 10 values from the min to the max 
gres = np.linspace(data['gre'].min(), data['gre'].max(), 10)
print gres
# array([ 220.        ,  284.44444444,  348.88888889,  413.33333333,
#         477.77777778,  542.22222222,  606.66666667,  671.11111111,
#         735.55555556,  800.        ])
gpas = np.linspace(data['gpa'].min(), data['gpa'].max(), 10)
print gpas
# array([ 2.26      ,  2.45333333,  2.64666667,  2.84      ,  3.03333333,
#         3.22666667,  3.42      ,  3.61333333,  3.80666667,  4.        ])


# enumerate all possibilities
combos = pd.DataFrame(cartesian([gres, gpas, [1, 2, 3, 4], [1.]]))
combos.columns = ['gre', 'gpa', 'prestige', 'intercept']

[ 220.          284.44444444  348.88888889  413.33333333  477.77777778
  542.22222222  606.66666667  671.11111111  735.55555556  800.        ]
[ 2.26        2.45333333  2.64666667  2.84        3.03333333  3.22666667
  3.42        3.61333333  3.80666667  4.        ]


#### 5.1 Recreate the dummy variables

In [70]:
# recreate the dummy variables

# keep only what we need for making predictions

new_dummies = pd.get_dummies(combos['prestige'], prefix ='prestige', drop_first= True)
new_df = combos.join(new_dummies).drop('prestige', axis =1)

new_df.head()

,gre,gpa,intercept,prestige_2.0,prestige_3.0,prestige_4.0
0,220.0,2.260000,1.0,0.0,0.0,0.0
1,220.0,2.260000,1.0,1.0,0.0,0.0
2,220.0,2.260000,1.0,0.0,1.0,0.0
3,220.0,2.260000,1.0,0.0,0.0,1.0
4,220.0,2.453333,1.0,0.0,0.0,0.0


#### 5.2 Make predictions on the enumerated dataset

In [73]:
column = ['gre', 'gpa', 'prestige_2.0', 'prestige_3.0', 'prestige_4.0']
new_df['prediction'] = score.predict(new_df[column])
new_df.head()

,gre,gpa,intercept,prestige_2.0,prestige_3.0,prestige_4.0,prediction
0,220.0,2.260000,1.0,0.0,0.0,0.0,0.500412
1,220.0,2.260000,1.0,1.0,0.0,0.0,0.277966
2,220.0,2.260000,1.0,0.0,1.0,0.0,0.177139
3,220.0,2.260000,1.0,0.0,0.0,1.0,0.133741
4,220.0,2.453333,1.0,0.0,0.0,0.0,0.494018


#### 5.3 Interpret findings for the last 4 observations

In [74]:
new_df.tail()

,gre,gpa,intercept,prestige_2.0,prestige_3.0,prestige_4.0,prediction
395,800.0,3.806667,1.0,0.0,0.0,1.0,0.217506
396,800.0,4.000000,1.0,0.0,0.0,0.0,0.637399
397,800.0,4.000000,1.0,1.0,0.0,0.0,0.403204
398,800.0,4.000000,1.0,0.0,1.0,0.0,0.274202
399,800.0,4.000000,1.0,0.0,0.0,1.0,0.213184


Answer: When holding gre and gpa constant the prestige of the school and the admission rate has a direct relationship. When the prestige of the school increases the admission probability increases as well.

## Bonus

Plot the probability of being admitted into graduate school, stratified by GPA and GRE score.